In [1]:
import numpy as np

class Layer:
    def __init__(self):
        pass

    def forward(self, input):
        return input

    def backward(self, input, grad_output):
        num_units = input.shape[1]
        d_layer_d_input = np.eye(num_units)
        return np.dot(grad_output, d_layer_d_input)
class ReLU(Layer):
    def __init__(self):
        pass

    def forward(self, input):
        return np.maximum(0,input)

    def backward(self, input, grad_output):
        relu_grad = input > 0
        return grad_output*relu_grad
class Dense(Layer):
    def __init__(self, input_units, output_units, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.weights = np.random.normal(loc=0.0, 
                                        scale = np.sqrt(2/(input_units+output_units)), 
                                        size = (input_units,output_units))
        self.biases = np.zeros(output_units)

    def forward(self,input):
        return np.dot(input,self.weights) + self.biases

    def backward(self,input,grad_output):
        grad_input = np.dot(grad_output, self.weights.T)
        grad_weights = np.dot(input.T, grad_output)
        grad_biases = grad_output.mean(axis=0)*input.shape[0]
        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases
        return grad_input
def softmax_crossentropy_with_logits(logits,reference_answers):
    logits_for_answers = logits[np.arange(len(logits)),reference_answers]
    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
    return xentropy

def grad_softmax_crossentropy_with_logits(logits,reference_answers):
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)),reference_answers] = 1
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    return (- ones_for_answers + softmax) / logits.shape[0]
class NeuralNetwork:
    def __init__(self, input_units, hidden_units, output_units, learning_rate=0.1):
        self.dense1 = Dense(input_units, hidden_units, learning_rate)
        self.activation1 = ReLU()
        self.dense2 = Dense(hidden_units, output_units, learning_rate)

    def forward(self, X):
        dense1_out = self.dense1.forward(X)
        activation1_out = self.activation1.forward(dense1_out)
        dense2_out = self.dense2.forward(activation1_out)
        return dense2_out

    def predict(self,X):
        logits = self.forward(X)
        return np.argmax(logits,axis=-1)

    def fit(self, X, y, epochs):
        for epoch in range(epochs):
            logits = self.forward(X)
            loss = softmax_crossentropy_with_logits(logits, y)
            loss_grad = grad_softmax_crossentropy_with_logits(logits, y)
            self.dense2.backward(self.activation1.forward(self.dense1.forward(X)), loss_grad)
            self.activation1.backward(self.dense1.forward(X), self.dense2.backward(self.activation1.forward(self.dense1.forward(X)), loss_grad))
            self.dense1.backward(X, self.activation1.backward(self.dense1.forward(X), self.dense2.backward(self.activation1.forward(self.dense1.forward(X)), loss_grad)))

            if epoch % 10 == 0:
                print(f'Epoch: {epoch}, Loss: {np.mean(loss)}')



In [2]:
import pandas as pd
df = pd.read_csv('Acoustic Features.csv')
def normalize(data, i):
    # Initialize a list to store the means of each column
    means = [0] * i
    # Initialize a list to store the standard deviations of each column
    stds = [1] * i
    # Loop through the columns
    for j in range(i):
        # Compute the mean of the column
        mean = data.iloc[:, j].mean()
        # Compute the standard deviation of the column
        std = data.iloc[:, j].std()
        # Store the mean and standard deviation
        means[j] = mean
        stds[j] = std
    # Loop through the rows
    for index, row in data.iterrows():
        # Loop through the columns
        for j in range(i):
            # Normalize the value by subtracting the mean and dividing by the standard deviation
            row[j] = (row[j] - means[j]) / stds[j]
    # Return the normalized data
    return data
# drop duplicate
df = df.drop_duplicates(keep='first')
# Split feature and target
X = df.drop(columns=['Class'])
y = df['Class']
y = pd.get_dummies(y, columns = ['Class'])
print(X.shape)
print(y.shape)

(388, 50)
(388, 4)


In [3]:
network = NeuralNetwork(X.shape[1], 5, y.shape[1])
network.fit(X, y, epochs=100)

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed